# ML / Nanoreactor Subgroup  21 Feb 2019 : 
# Building a NN with Keras - Handwritten Digit Recognition 

## Key steps in building a Neural Network:

1. Data loading / Preprocessing 
2. Define the model architecture
3. Compile the model
4. Train it
5. Evaluate the trained model
6. Using neural networks in anger

## Useful links:

 - https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/
 
 - https://keras.io/getting-started/sequential-model-guide/ 
 
 - https://www.dataweekends.com/blog/2017/03/09/set-up-your-mac-for-deep-learning-with-python-keras-and-tensorflow 
 
 - http://cs231n.stanford.edu/

Contact: Hayley Weir, Keiran Thompson

In [ ]:
import keras, tensorflow
%pylab inline

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard
K.set_image_dim_ordering('th')

## Baseline Single Hidden Layer Model

### 1. Data loading  and Preprocessing

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# plot some images
n = 59999
plt.imshow(X_train[n], cmap=plt.get_cmap('gray'))
print("Label: ", y_train[n])
plt.show()

In [ ]:
X_train.shape

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [ ]:
# flatten 28*28 images to a 784 vector for each image
num_pixels = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
# one hot encode outputs (multi-class classification problem)
y_train_1hot = np_utils.to_categorical(y_train)
y_test_1hot = np_utils.to_categorical(y_test)
num_classes = y_test_1hot.shape[1]

### 2. Define the model architecture

In [ ]:
# multilayer perceptron model 
# create model (one hidden layer)
model = Sequential()
model.add(Dense(num_pixels, input_dim=num_pixels, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
print(model.summary())

### 3. Compile the model

In [ ]:
# specify the loss function, optimizer, and metrics to train against. 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 4. Train it

In [ ]:
# Configure logging
tensorboard = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=True, write_images=False)
# Fit the model
model.fit(X_train, y_train_1hot, validation_split=0.1, epochs=10, batch_size=256, verbose=1, callbacks=[tensorboard])

### 5. What did we get?
Use tensorboard to explore the distribution of 
```
$ tensorboard --logdir=logs
```

### 6. Evaluate the Model

In [ ]:
# evaluation of the model using test set
scores = model.evaluate(X_test, y_test_1hot, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

### 7. Predictions

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)

In [ ]:
# pick a random entry from the test set
n = np.random.randint(X_test.shape[0], size=1)
print("Prediction: ", predictions[n])
if predictions[n] == y_test[n]:
    print("Correct!!")
else:
    print("Incorrect, uh oh!")
plt.imshow(X_test[n].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.show()

## Convolutional Neural Network Model

### 1. Data loading  and Preprocessing

In [ ]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [ ]:
X_train.shape

In [ ]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train_1hot = np_utils.to_categorical(y_train)
y_test_1hot = np_utils.to_categorical(y_test)
num_classes = y_test_1hot.shape[1]

### 2. Define the model architecture

In [ ]:
# create a new model, add some convolutions
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
print(model.summary())

### 3. Compile the model

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# configure logging so we can visualise the model with tensorboard
tensorboard = TensorBoard(log_dir='logs', histogram_freq=0,
                          write_graph=True, write_images=False)

### 4. Train it

In [ ]:
# Fit the model
model.fit(X_train, y_train_1hot, epochs=10, validation_split=0.1, batch_size=128, verbose=1,
         shuffle=True, callbacks=[tensorboard])

### 6. Evaluate the Model

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test_1hot, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

### 7. Predictions

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)

In [ ]:
# pick a random entry from the test set
n = np.random.randint(X_test.shape[0], size=1)
print("Prediction: ", predictions[n])
if predictions[n] == y_test[n]:
    print("Correct!!")
else:
    print("Incorrect, uh oh!")
plt.imshow(X_test[n].reshape(28,28), cmap=plt.get_cmap('gray'))
plt.show()